In [39]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from dateutil import parser

In [40]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

url = "https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=2"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

body = soup.find('ul', {"class": "list-news indeks-new"})

if body is not None:
    articles = body.find_all('li')

    for article in articles:
        link_tag = article.find('a')
        
        # Check if the 'a' tag is found
        if link_tag is not None:
            link = link_tag.get('href')
            
            # Check if the 'href' attribute is present
            if link is not None:
                print(link)

https://finansial.bisnis.com/read/20231101/11/1710296/sri-mulyani-wanti-wanti-dampak-pelemahan-rupiah-ke-neraca-keuangan-perusahaan
https://ekonomi.bisnis.com/read/20231101/257/1710261/kemenperin-sebut-industri-manufaktur-tertekan-imbas-hgbt-tak-berjalan-lancar
https://market.bisnis.com/read/20231101/7/1710221/laju-saham-transportasi-idxtrans-melandai-apa-sebabnya
https://semarang.bisnis.com/read/20231101/535/1710300/hingga-september-75952-wisatawan-mancanegara-pelesir-ke-diy
https://ekonomi.bisnis.com/read/20231101/44/1710286/skk-migas-pembeli-gas-blok-masela-sudah-oversubscribed
https://finansial.bisnis.com/read/20231101/563/1710298/fenomena-teman-pinjam-akun-pinjol-begini-kata-ojk
https://ekonomi.bisnis.com/read/20231101/257/1710262/cukai-tinggi-produksi-industri-rokok-diramal-turun-10
https://www.bisnis.com/read/20231101/638/1710274/mahfud-md-ogah-tanggapi-pencopotan-baliho-kampanye-di-bali
https://ekonomi.bisnis.com/read/20231101/47/1710301/alam-sutera-tawarkan-hunian-mewah-klaste

In [41]:
def scrape_links(date,page_number):
    format=datetime.strptime(date, "%Y-%m-%d")
# print(format)
    formatted_date_string = format.strftime("%Y-%m-%d")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.bisnis.com/index/page/?c=0&d={formatted_date_string}&per_page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.find('ul', {"class": "list-news indeks-new"})
    links=[]
    if body is not None:
        articles = body.find_all('li')

        for article in articles:
            link_tag = article.find('a')
            
            # Check if the 'a' tag is found
            if link_tag is not None:
                link = link_tag.get('href')
                
                # Check if the 'href' attribute is present
                if link is not None:
                    links.append(link)
    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    
    return links

In [42]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 1
    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 1

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [43]:
link=scrape_link_per_day('2023-11-01')

Scraped 15 links from page 1 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=1
Scraped 15 links from page 2 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=2
Scraped 15 links from page 3 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=3
Scraped 15 links from page 4 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=4
Scraped 15 links from page 5 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=5
Scraped 15 links from page 6 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=6
Scraped 15 links from page 7 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=7
Scraped 15 links from page 8 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=8
Scraped 15 links from page 9 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=9
Scraped 15 links from page 10 url https://www.bisnis.com/index/page/?c=0&d=2023-11-01&per_page=10
Scraped 15 links from page 11 url http

In [52]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"class": "detailsTitleCaption"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"  
                    # tanggal berita
                    date_elem = soup.find('div', {"class": "detailsAttributeDates"})
                    # print(date_elem)
                    if date_elem:
                        date_text= date_elem.text.strip()
                        day_names_mapping = {
                            'Senin': 'Monday',
                            'Selasa': 'Tuesday',
                            'Rabu': 'Wednesday',
                            'Kamis': 'Thursday',
                            'Jumat': 'Friday',
                            'Sabtu': 'Saturday',
                            'Minggu': 'Sunday'
                        }

                        # Replace the Indonesian day name with English day name
                        for indonesian_day, english_day in day_names_mapping.items():
                            date_text = date_text.replace(indonesian_day, english_day)

                        # Convert the date string to a datetime object
                        date_object = datetime.strptime(date_text, '%A, %d %B %Y | %H:%M')

                        # Format the datetime object to the desired format
                        formatted_date = date_object.strftime('%Y-%m-%d')
                        # date_text = date_text
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('article', {"class": "detailsContent force-17 mt40"})
                        
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                                content_text += p.text.strip() + "\n"
                            
                        if content_text.strip():
                            content_text=content_text
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text="Content not found"
                    else:
                            content_text="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [53]:
url='https://ekonomi.bisnis.com/read/20231101/257/1710302/inflasi-oktober-capai-256-pengusaha-minta-pemerintah-kendalikan-pelemahan-rupiah'
data_bisnis = scrape_url(url)

In [54]:
data_bisnis

{'title': 'Inflasi Oktober Capai 2,56%, Pengusaha Minta Pemerintah Kendalikan Pelemahan Rupiah',
 'date': '2023-11-01',
 'content': 'Bisnis.com, JAKARTA - Asosiasi Pengusaha Indonesia (Apindo) meminta pemerintah untuk bertindak cepat dalam mengendalikan pelemahan rupiah guna menjaga tingkat inflasi tetap stabil.Ketua Umum Apindo, Shinta W. Kamdani berharap pemerintah dapat segera meningkatkan efektivitas instrumen-instrumen dan melakukan intervensi kebijakan atas pengendalian pelemahan rupiah.Lebih lanjut, Shinta memberikan beberapa cara yang dapat dilakukan yakni dengan stimulasi ekspor, peningkatan konsistensi implementasi kebijakan reformasi struktural iklim usaha untuk investasi asing, peningkatan fasilitasi investasi asing.Tak kalah penting, kampanye penggunaan Local Currency Settlement (LCS), dan perbaikan daya saing sektor perbankan nasional sebagai parameter suku bunga simpanan dalam mata uang asing untuk mendongkrak penempatan DHE di dalam negeri secara sukarela, dan lainnya."